In [1]:
from dotenv import load_dotenv
from snowflake.snowpark.session import Session
import os

load_dotenv()

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
}

snowpark_session = Session.builder.configs(connection_parameters).create()

In [22]:
snowpark_session.close()

In [ ]:
from snowflake.cortex import Complete

print(Complete("mistral-large2", "can you understand images ?"))

In [3]:
import os
from snowflake.core import Root
from typing import List


class CortexSearchRetriever:

    def __init__(self, snowpark_session: Session, limit_to_retrieve: int = 3):
        self._snowpark_session = snowpark_session
        self._limit_to_retrieve = limit_to_retrieve

    def retrieve(self, query: str) -> List[str]:
        root = Root(self._snowpark_session)
        cortex_search_service = (
            root.databases[os.getenv("SNOWFLAKE_DATABASE")]
            .schemas[os.getenv("SNOWFLAKE_SCHEMA")]
            .cortex_search_services["CC_SEARCH_SERVICE_CS"]
        )
        resp = cortex_search_service.search(
            query=query,
            columns=["TEXT_CONTENT"],
            limit=self._limit_to_retrieve,
        )

        if resp.results:
            return [curr["TEXT_CONTENT"] for curr in resp.results]
        else:
            return []

In [4]:
retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

retrieved_context = retriever.retrieve(query="how was my christmas on December 25, 2024?")


In [ ]:
retrieved_context

In [ ]:
from trulens.core import TruSession
from trulens.connectors.snowflake import SnowflakeConnector

tru_snowflake_connector = SnowflakeConnector(snowpark_session=snowpark_session)

tru_session = TruSession(connector=tru_snowflake_connector)

In [ ]:
from trulens.apps.custom import instrument


class RAG_from_scratch:

    def __init__(self):
        self.retriever = CortexSearchRetriever(snowpark_session=snowpark_session, limit_to_retrieve=4)

    @instrument
    def retrieve_context(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        return self.retriever.retrieve(query)

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        prompt = f"""You are a personal AI assistant who helps the user recall and elaborate on their past thoughts, plans, and discussions.
        You have access to the user's personal notes and memories.

        Context of previous discussions:
        <context>
        {context_str}
        </context>

        User's current question: {query}

        Based on the context and your understanding, provide a helpful and precise response.
        If the context directly addresses the question, use those details.
        If not, respond based on the most relevant information available.
        Always be supportive and sound like a trusted personal assistant.

        Respond with a clear, natural text response. Do not use any special formatting or JSON structure.
        """
        return Complete("mistral-large2", prompt)

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve_context(query)
        return self.generate_completion(query, context_str)


rag = RAG_from_scratch()

In [ ]:
from trulens.providers.cortex.provider import Cortex
from trulens.core import Feedback
from trulens.core import Select
import numpy as np

provider = Cortex(
    snowpark_session,
    model_engine="mistral-large2",
)

f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
    .on(Select.RecordCalls.retrieve_context.rets[:].collect())
    .on_output()
)

f_context_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on_input()
    .on(Select.RecordCalls.retrieve_context.rets[:])
    .aggregate(np.mean)
)

f_answer_relevance = (
    Feedback(provider.relevance, name="Answer Relevance")
    .on_input()
    .on_output()
    .aggregate(np.mean)
)

In [ ]:
from trulens.apps.custom import TruCustomApp

tru_rag = TruCustomApp(
    rag,
    app_name="memex",
    app_version="simple",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
    )

In [15]:
prompts = ["how was my christmas this year ?", "how was my last saturday"]

In [ ]:
with tru_rag as recording:
  for prompt in prompts:
        rag.query(prompt)

tru_session.get_leaderboard()
